# 1. Raw Data

### 1.1 Create DB Conncection

In [4]:
import db_connect
sql = db_connect.DatabaseConnect('CS581GROUP6','localhost','root','qwerty','3306')

# 2. Preprocessing

In [5]:
import numpy as np
from datetime import datetime  
from datetime import timedelta  

### 2.1 Mapping Coordinates to Destination ID

In [25]:
def getdestid(lat,longt):
   return df_destinations.iloc[df_destinations.apply(lambda x: np.linalg.norm(np.array([lat, longt]) - np.array([x['destLat'],x['destLong']])), axis=1).idxmin(axis = 0)]['destID']

In [26]:
from datetime import datetime
import numpy as np
import pandas as pd

df = pd.read_csv('RideSharingData/filtered_data.csv')
df_destinations = sql.query('select * from destination',True)

df['tpep_pickup_datetime'] = [datetime.strptime(pickup_time, '%Y-%m-%d %H:%M:%S') 
                                          for pickup_time in df['tpep_pickup_datetime']]

df['tpep_dropoff_datetime'] = [datetime.strptime(dropoff_time, '%Y-%m-%d %H:%M:%S') 
                                          for dropoff_time in df['tpep_dropoff_datetime']]
#df['destination_id'] = df.apply(getdestid, index=['dropoff_latitude', 'dropoff_longitude'])

#df['destination_id'] = getdestid(df['dropoff_latitude'],df['dropoff_longitude'])

#df['cost'] = cost_function(df['trip_distance'], df['tpep_dropoff_datetime']-df['tpep_pickup_datetime'], False)

In [ ]:
df['destination_id'] = df.iloc[:].apply(lambda x: getdestid(x['dropoff_latitude'],x['dropoff_longitude']),axis = 1)

In [29]:
#df['destination_id']

### 2.2 Pool Window Assignment

In [10]:
def getpoolingwindow(startTime, CurrentTime, counter):
    if(CurrentTime < startTime + timedelta(minutes = 5)):
        return counter, startTime
    else:
        while((current - startTime).seconds > 300):
            startTime += timedelta(minutes = 5)
        
        return counter+1, startTime

### 2.3 Calculate Delay

In [11]:
def getDelay(pickupTime, dropoffTime):
    tripTime = (dropoffTime - pickupTime).seconds
    tenpercent = tripTime/4
    return min(tenpercent, 600)

### 2.4 Calculate Walking Time

In [12]:
def getWalkingTime(pickupTime, dropoffTime):
    tripTime = (dropoffTime - pickupTime).seconds
    tenpercent = tripTime/10
    return min(tenpercent, 240)

### 2.5 Calculate Cost for Individual Ride

In [6]:
def millisec_to_min(milli_time):
    return milli_time/60000

In [7]:
def meter_to_mile(meter_distance):
    return meter_distance/1609.34

In [8]:
def cost_function(distance, time, isSharedTrip):
    #Convert milliseconds to minutes
    time = millisec_to_min(time)
    #Convert meters to miles
    distance = meter_to_mile(distance)
    if isSharedTrip:
        cost_per_mile = 1.00
        cost_per_min = 0.18
    else:
        cost_per_mile = 1.80
        cost_per_min = 0.28
    base_fare = 0
    booking_fee = 2.3
    min_fare = 7.3
    
    total_fare = (cost_per_mile * distance) + (cost_per_min * time) + base_fare + booking_fee + min_fare
    return total_fare

### 2.6 Assign Request ID

In [ ]:
walkingTime =  getDelay(pickupTime, dropoffTime)
delayTime= getWalkingTime(pickupTime, dropoffTime)  
poolID, startTime = getpoolingwindow(startTime, current, 10)
destID = getdestid(latlong)

# 3. Df_Request Table